In [74]:
import pandas as pd
video_duration_list = pd.read_csv("videoduration.csv")
video_duration_list.columns
#make a dictionary of name : duration columns
video_duration_dict = dict(zip(video_duration_list['name'], video_duration_list['duration']))
video_duration_dict

{'2024-11-3015-02-20.csv': 446,
 '2024-11-3015-14-06.csv': 1129,
 '2024-11-3015-52-32.csv': 662,
 '2024-11-3016-07-16.csv': 436,
 '2024-12-0120-22-43.csv': 1387,
 '2024-12-0121-35-34.csv': 1156,
 '2024-12-0121-56-31.csv': 466,
 '2024-12-0122-54-33.csv': 533,
 '2024-12-0202-31-45.csv': 489,
 '2024-12-0202-46-38.csv': 694,
 '2024-12-0214-55-34.csv': 910,
 '2024-12-0218-58-24.csv': 6,
 '2024-12-0218-58-32.csv': 3041,
 '2024-12-0220-04-44.csv': 509,
 '2024-12-0220-15-35.csv': 407,
 '2024-12-0220-23-59.csv': 554,
 '2024-12-0220-23-15.csv': 2578,
 '2024-12-1121-14-21.csv': 621,
 '2024-12-1121-26-44.csv': 461,
 '2024-12-1121-41-00.csv': 554,
 '2024-12-1121-54-20.csv': 456,
 '2024-12-1122-05-27.csv': 442,
 '2024-12-1122-45-12.csv': 409,
 '2024-12-1123-10-00.csv': 418,
 '2024-12-1123-21-51.csv': 457,
 '2024-12-0319-29-48.csv': 5195,
 '2024-12-1118-26-33.csv': 358,
 '2024-12-1118-45-08.csv': 363,
 '2024-12-1118-54-55.csv': 368,
 '2024-12-1119-06-43.csv': 368,
 '2024-12-1119-16-53.csv': 390}

In [75]:
import json
from datetime import datetime, timedelta
import os
import pandas as pd

def extract_video_data_between_timestamps(video_files, start_timestamp, end_timestamp):
    """
    Extract data entries from a series of video files between two timestamps.

    :param video_files: Dictionary of video file names and their durations in seconds
                       {"video_name.csv": duration_in_seconds}
    :param start_timestamp: Start timestamp as a string in the format '%Y-%m-%d %H-%M-%S'
    :param end_timestamp: End timestamp as a string in the format '%Y-%m-%d %H-%M-%S'
    :return: Dictionary with video file names and corresponding filtered DataFrames
    """
    start_dt = datetime.strptime(start_timestamp, '%Y-%m-%d %H-%M-%S')
    end_dt = datetime.strptime(end_timestamp, '%Y-%m-%d %H-%M-%S')

    filtered_data = {}

    for video_name, duration in video_files.items():
        # Extract the start time from the video name
        video_start_str = video_name.split('.')[0]  # Remove file extension
        video_start_dt = datetime.strptime(video_start_str, '%Y-%m-%d%H-%M-%S')
        video_end_dt = video_start_dt + timedelta(seconds=duration)

        # Check if the video overlaps with the desired time range
        if video_end_dt < start_dt or video_start_dt > end_dt:
            continue  # Skip videos outside the range

        # Load the video data
        video_path = f'./Out/{video_name}'
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"The video file '{video_name}' was not found.")

        video_data = pd.read_csv(video_path)

        # If timestamp is in seconds relative to video start, use it directly
        video_data['absolute_timestamp'] = video_start_dt + pd.to_timedelta(video_data[' timestamp'], unit='s')

        # Filter rows within the desired time range
        filtered_video_data = video_data[(video_data['absolute_timestamp'] >= start_dt) &
                                         (video_data['absolute_timestamp'] <= end_dt)]

        filtered_data[video_name] = filtered_video_data

    return filtered_data

# Example usage:
video_files = video_duration_dict

start_time = '2024-12-01 22-55-00'
end_time = '2024-12-01 23-02-00'

filtered_videos = extract_video_data_between_timestamps(video_files, start_time, end_time)

for video_name, data in filtered_videos.items():
    print(f"Filtered data for {video_name}:")
    print(data.head())
    print(data.tail())

# Concat all data
all_au_data = pd.concat(filtered_videos.values())

Filtered data for 2024-12-0122-54-33.csv:
     frame   face_id   timestamp  ...   AU28_c   AU45_c      absolute_timestamp
810    811         0      27.000  ...      0.0      0.0 2024-12-01 22:55:00.000
811    812         0      27.033  ...      0.0      0.0 2024-12-01 22:55:00.033
812    813         0      27.067  ...      0.0      0.0 2024-12-01 22:55:00.067
813    814         0      27.100  ...      0.0      0.0 2024-12-01 22:55:00.100
814    815         0      27.133  ...      0.0      0.0 2024-12-01 22:55:00.133

[5 rows x 715 columns]
       frame   face_id   timestamp  ...   AU28_c   AU45_c      absolute_timestamp
13406  13407         0     446.867  ...      1.0      1.0 2024-12-01 23:01:59.867
13407  13408         0     446.900  ...      1.0      1.0 2024-12-01 23:01:59.900
13408  13409         0     446.933  ...      1.0      1.0 2024-12-01 23:01:59.933
13409  13410         0     446.967  ...      1.0      1.0 2024-12-01 23:01:59.967
13410  13411         0     447.000  ...     

In [ ]:
type(filtered_videos)
#convert to df
au_features_df = all_au_data
au_features_df

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,gaze_1_z,gaze_angle_x,gaze_angle_y,eye_lmk_x_0,eye_lmk_x_1,eye_lmk_x_2,eye_lmk_x_3,eye_lmk_x_4,eye_lmk_x_5,eye_lmk_x_6,eye_lmk_x_7,eye_lmk_x_8,eye_lmk_x_9,eye_lmk_x_10,eye_lmk_x_11,eye_lmk_x_12,eye_lmk_x_13,eye_lmk_x_14,eye_lmk_x_15,eye_lmk_x_16,eye_lmk_x_17,eye_lmk_x_18,eye_lmk_x_19,eye_lmk_x_20,eye_lmk_x_21,eye_lmk_x_22,eye_lmk_x_23,eye_lmk_x_24,eye_lmk_x_25,eye_lmk_x_26,eye_lmk_x_27,eye_lmk_x_28,eye_lmk_x_29,eye_lmk_x_30,eye_lmk_x_31,eye_lmk_x_32,eye_lmk_x_33,eye_lmk_x_34,eye_lmk_x_35,eye_lmk_x_36,eye_lmk_x_37,eye_lmk_x_38,eye_lmk_x_39,eye_lmk_x_40,eye_lmk_x_41,eye_lmk_x_42,eye_lmk_x_43,eye_lmk_x_44,eye_lmk_x_45,eye_lmk_x_46,eye_lmk_x_47,eye_lmk_x_48,eye_lmk_x_49,eye_lmk_x_50,eye_lmk_x_51,eye_lmk_x_52,eye_lmk_x_53,eye_lmk_x_54,eye_lmk_x_55,eye_lmk_y_0,eye_lmk_y_1,eye_lmk_y_2,eye_lmk_y_3,eye_lmk_y_4,eye_lmk_y_5,eye_lmk_y_6,eye_lmk_y_7,eye_lmk_y_8,eye_lmk_y_9,eye_lmk_y_10,eye_lmk_y_11,eye_lmk_y_12,eye_lmk_y_13,eye_lmk_y_14,eye_lmk_y_15,eye_lmk_y_16,eye_lmk_y_17,eye_lmk_y_18,eye_lmk_y_19,eye_lmk_y_20,eye_lmk_y_21,eye_lmk_y_22,eye_lmk_y_23,eye_lmk_y_24,eye_lmk_y_25,eye_lmk_y_26,eye_lmk_y_27,eye_lmk_y_28,eye_lmk_y_29,eye_lmk_y_30,eye_lmk_y_31,eye_lmk_y_32,eye_lmk_y_33,eye_lmk_y_34,eye_lmk_y_35,eye_lmk_y_36,eye_lmk_y_37,eye_lmk_y_38,eye_lmk_y_39,eye_lmk_y_40,eye_lmk_y_41,eye_lmk_y_42,eye_lmk_y_43,eye_lmk_y_44,eye_lmk_y_45,eye_lmk_y_46,eye_lmk_y_47,eye_lmk_y_48,eye_lmk_y_49,eye_lmk_y_50,eye_lmk_y_51,eye_lmk_y_52,eye_lmk_y_53,eye_lmk_y_54,eye_lmk_y_55,eye_lmk_X_0,eye_lmk_X_1,eye_lmk_X_2,eye_lmk_X_3,eye_lmk_X_4,eye_lmk_X_5,eye_lmk_X_6,eye_lmk_X_7,eye_lmk_X_8,eye_lmk_X_9,eye_lmk_X_10,eye_lmk_X_11,eye_lmk_X_12,eye_lmk_X_13,eye_lmk_X_14,eye_lmk_X_15,eye_lmk_X_16,eye_lmk_X_17,eye_lmk_X_18,eye_lmk_X_19,eye_lmk_X_20,eye_lmk_X_21,eye_lmk_X_22,eye_lmk_X_23,eye_lmk_X_24,eye_lmk_X_25,eye_lmk_X_26,eye_lmk_X_27,eye_lmk_X_28,eye_lmk_X_29,eye_lmk_X_30,eye_lmk_X_31,eye_lmk_X_32,eye_lmk_X_33,eye_lmk_X_34,eye_lmk_X_35,eye_lmk_X_36,eye_lmk_X_37,eye_lmk_X_38,eye_lmk_X_39,eye_lmk_X_40,eye_lmk_X_41,eye_lmk_X_42,eye_lmk_X_43,eye_lmk_X_44,eye_lmk_X_45,eye_lmk_X_46,eye_lmk_X_47,eye_lmk_X_48,eye_lmk_X_49,eye_lmk_X_50,eye_lmk_X_51,eye_lmk_X_52,eye_lmk_X_53,eye_lmk_X_54,eye_lmk_X_55,eye_lmk_Y_0,eye_lmk_Y_1,eye_lmk_Y_2,eye_lmk_Y_3,eye_lmk_Y_4,eye_lmk_Y_5,eye_lmk_Y_6,eye_lmk_Y_7,eye_lmk_Y_8,eye_lmk_Y_9,eye_lmk_Y_10,eye_lmk_Y_11,eye_lmk_Y_12,eye_lmk_Y_13,eye_lmk_Y_14,eye_lmk_Y_15,eye_lmk_Y_16,eye_lmk_Y_17,eye_lmk_Y_18,eye_lmk_Y_19,eye_lmk_Y_20,eye_lmk_Y_21,eye_lmk_Y_22,eye_lmk_Y_23,eye_lmk_Y_24,eye_lmk_Y_25,eye_lmk_Y_26,eye_lmk_Y_27,eye_lmk_Y_28,eye_lmk_Y_29,eye_lmk_Y_30,eye_lmk_Y_31,eye_lmk_Y_32,eye_lmk_Y_33,eye_lmk_Y_34,eye_lmk_Y_35,eye_lmk_Y_36,eye_lmk_Y_37,eye_lmk_Y_38,eye_lmk_Y_39,eye_lmk_Y_40,eye_lmk_Y_41,eye_lmk_Y_42,eye_lmk_Y_43,eye_lmk_Y_44,eye_lmk_Y_45,eye_lmk_Y_46,eye_lmk_Y_47,eye_lmk_Y_48,eye_lmk_Y_49,eye_lmk_Y_50,eye_lmk_Y_51,eye_lmk_Y_52,eye_lmk_Y_53,eye_lmk_Y_54,eye_lmk_Y_55,eye_lmk_Z_0,eye_lmk_Z_1,eye_lmk_Z_2,eye_lmk_Z_3,eye_lmk_Z_4,eye_lmk_Z_5,eye_lmk_Z_6,eye_lmk_Z_7,eye_lmk_Z_8,eye_lmk_Z_9,eye_lmk_Z_10,eye_lmk_Z_11,eye_lmk_Z_12,eye_lmk_Z_13,eye_lmk_Z_14,eye_lmk_Z_15,eye_lmk_Z_16,eye_lmk_Z_17,eye_lmk_Z_18,eye_lmk_Z_19,eye_lmk_Z_20,eye_lmk_Z_21,eye_lmk_Z_22,eye_lmk_Z_23,eye_lmk_Z_24,eye_lmk_Z_25,eye_lmk_Z_26,eye_lmk_Z_27,eye_lmk_Z_28,eye_lmk_Z_29,eye_lmk_Z_30,eye_lmk_Z_31,eye_lmk_Z_32,eye_lmk_Z_33,eye_lmk_Z_34,eye_lmk_Z_35,eye_lmk_Z_36,eye_lmk_Z_37,eye_lmk_Z_38,eye_lmk_Z_39,eye_lmk_Z_40,eye_lmk_Z_41,eye_lmk_Z_42,eye_lmk_Z_43,eye_lmk_Z_44,eye_lmk_Z_45,eye_lmk_Z_46,eye_lmk_Z_47,eye_lmk_Z_48,eye_lmk_Z_49,eye_lmk_Z_50,eye_lmk_Z_51,eye_lmk_Z_52,eye_lmk_Z_53,eye_lmk_Z_54,eye_lmk_Z_55,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,x_12,x_13,x_14,x_15,x_16,x_17,x_18,x_19,x_20,x_21,x_22,x_23,x_24,x_25,x_26,x_27,x_28,x_29,x_30,x_31,x_32,x_33,x_34,x_35,x_36,x_37,x_38,x_39,x_40,x_41,x_42,x_43,x_44,x_45,x_46,x_47,x_48,x_4

In [ ]:
au_features_df['absolute_timestamp']

810     2024-12-01 22:55:00.000
811     2024-12-01 22:55:00.033
812     2024-12-01 22:55:00.067
813     2024-12-01 22:55:00.100
814     2024-12-01 22:55:00.133
                  ...          
13406   2024-12-01 23:01:59.867
13407   2024-12-01 23:01:59.900
13408   2024-12-01 23:01:59.933
13409   2024-12-01 23:01:59.967
13410   2024-12-01 23:02:00.000
Name: absolute_timestamp, Length: 12601, dtype: datetime64[ns]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assume `merged_df` is the dataframe after merging AU and HR data and aggregating filtered_videos on seconds absolute timestamp
au_features_df.set_index("absolute_timestamp", inplace=True)  
selected_aus = [' AU01_r', ' AU02_r', ' AU04_r', ' AU05_r', ' AU06_r',
               ' AU07_r', ' AU09_r', ' AU10_r', ' AU12_r', ' AU14_r',
               ' AU15_r', ' AU17_r', ' AU20_r', ' AU23_r', ' AU25_r',
               ' AU26_r', ' AU45_r', ' AU01_c', ' AU02_c',
               ' AU04_c', ' AU05_c', ' AU06_c', ' AU07_c', ' AU09_c',
               ' AU10_c', ' AU12_c', ' AU14_c', ' AU15_c', ' AU17_c',
               ' AU20_c', ' AU23_c', ' AU25_c', ' AU26_c', ' AU28_c',
               ' AU45_c', ' pose_Rx', ' pose_Ry', ' pose_Rz', ' gaze_angle_x', ' gaze_angle_y']
  
au_aggregated = au_features_df[selected_aus].resample('1S').agg(['mean', 'max', 'std'])
au_aggregated

AU01_r                    AU02_r                  \
                       mean   max       std      mean   max       std   
absolute_timestamp                                                      
2024-12-01 22:55:00   0.000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:01   0.000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:02   0.000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:03   0.000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:04   0.000  0.00  0.000000  0.000000  0.00  0.000000   
...                     ...   ...       ...       ...   ...       ...   
2024-12-01 23:01:56   0.015  0.08  0.028009  0.015000  0.15  0.042325   
2024-12-01 23:01:57   0.000  0.00  0.000000  0.001000  0.01  0.003051   
2024-12-01 23:01:58   0.000  0.00  0.000000  0.105667  0.74  0.203481   
2024-12-01 23:01:59   0.000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 23:02:00   0.000  0.00       NaN  0.000000  0.00       NaN   

                       AU04_r                  AU05_r                  \
                         mean   max       std    mean   max       std   
absolute_timestamp                                                      
2024-12-01 22:55:00  0.000000  0.00  0.000000   0.000  0.00  0.000000   
2024-12-01 22:55:01  0.000000  0.00  0.000000   0.000  0.00  0.000000   
2024-12-01 22:55:02  0.000000  0.00  0.000000   0.000  0.00  0.000000   
2024-12-01 22:55:03  0.000000  0.00  0.000000   0.000  0.00  0.000000   
2024-12-01 22:55:04  0.000000  0.00  0.000000   0.000  0.00  0.000000   
...                       ...   ...       ...     ...   ...       ...   
2024-12-01 23:01:56  0.389333  0.98  0.353484   0.000  0.00  0.000000   
2024-12-01 23:01:57  0.061333  0.22  0.068165   0.052  0.45  0.121921   
2024-12-01 23:01:58  0.233000  0.94  0.361674   0.000  0.00  0.000000   
2024-12-01 23:01:59  0.772333  1.03  0.109409   0.000  0.00  0.000000   
2024-12-01 23:02:00  0.790000  0.79       NaN   0.000  0.00       NaN   

                       AU06_r                    AU07_r                  \
                         mean   max       std      mean   max       std   
absolute_timestamp                                                        
2024-12-01 22:55:00  0.000000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:01  0.000000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:02  0.000000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:03  0.000000  0.00  0.000000  0.000000  0.00  0.000000   
2024-12-01 22:55:04  0.000000  0.00  0.000000  0.000000  0.00  0.000000   
...                       ...   ...       ...       ...   ...       ...   
2024-12-01 23:01:56  0.004000  0.04  0.012205  2.521000  3.49  0.483560   
2024-12-01 23:01:57  0.001000  0.01  0.003051  2.691000  3.34  0.381235   
2024-12-01 23:01:58  0.081333  0.53  0.153550  2.727333  3.54  0.411431   
2024-12-01 23:01:59  0.158333  0.51  0.205964  3.711333  4.54  0.567048   
2024-12-01 23:02:00  0.000000  0.00       NaN  2.690000  2.69       NaN   

                       AU09_r                  AU10_r              AU12_r  \
                         mean   max       std    mean  max  std      mean   
absolute_timestamp                                                          
2024-12-01 22:55:00  0.000000  0.00  0.000000     0.0  0.0  0.0  0.000000   
2024-12-01 22:55:01  0.000000  0.00  0.000000     0.0  0.0  0.0  0.000000   
2024-12-01 22:55:02  0.000000  0.00  0.000000     0.0  0.0  0.0  0.000000   
2024-12-01 22:55:03  0.000000  0.00  0.000000     0.0  0.0  0.0  0.000000   
2024-12-01 22:55:04  0.000000  0.00  0.000000     0.0  0.0  0.0  0.000000   
...                       ...   ...       ...     ...  ...  ...       ...   
2024-12-01 23:01:56  0.365333  0.62  0.153212     0.0  0.0  0.0  0.292000   
2024-12-01 23:01:57  0.041333  0.32  0.081017     0.0  0.0  0.0  0.413333   
2024-12-01 23:01:58  0.404000  0.79  0.251870     0.0  0.0  0.0  0.531000   


In [ ]:
au_aggregated.columns = ['_'.join(col).strip() for col in au_aggregated.columns.values]
au_aggregated.reset_index(inplace=True)
merged_df = au_aggregated
merged_df

,absolute_timestamp,AU01_r_mean,AU01_r_max,AU01_r_std,AU02_r_mean,AU02_r_max,AU02_r_std,AU04_r_mean,AU04_r_max,AU04_r_std,AU05_r_mean,AU05_r_max,AU05_r_std,AU06_r_mean,AU06_r_max,AU06_r_std,AU07_r_mean,AU07_r_max,AU07_r_std,AU09_r_mean,AU09_r_max,AU09_r_std,AU10_r_mean,AU10_r_max,AU10_r_std,AU12_r_mean,AU12_r_max,AU12_r_std,AU14_r_mean,AU14_r_max,AU14_r_std,AU15_r_mean,AU15_r_max,AU15_r_std,AU17_r_mean,AU17_r_max,AU17_r_std,AU20_r_mean,AU20_r_max,AU20_r_std,AU23_r_mean,AU23_r_max,AU23_r_std,AU25_r_mean,AU25_r_max,AU25_r_std,AU26_r_mean,AU26_r_max,AU26_r_std,AU45_r_mean,AU45_r_max,AU45_r_std,AU01_c_mean,AU01_c_max,AU01_c_std,AU02_c_mean,AU02_c_max,AU02_c_std,AU04_c_mean,AU04_c_max,AU04_c_std,AU05_c_mean,AU05_c_max,AU05_c_std,AU06_c_mean,AU06_c_max,AU06_c_std,AU07_c_mean,AU07_c_max,AU07_c_std,AU09_c_mean,AU09_c_max,AU09_c_std,AU10_c_mean,AU10_c_max,AU10_c_std,AU12_c_mean,AU12_c_max,AU12_c_std,AU14_c_mean,AU14_c_max,AU14_c_std,AU15_c_mean,AU15_c_max,AU15_c_std,AU17_c_mean,AU17_c_max,AU17_c_std,AU20_c_mean,AU20_c_max,AU20_c_std,AU23_c_mean,AU23_c_max,AU23_c_std,AU25_c_mean,AU25_c_max,AU25_c_std,AU26_c_mean,AU26_c_max,AU26_c_std,AU28_c_mean,AU28_c_max,AU28_c_std,AU45_c_mean,AU45_c_max,AU45_c_std,pose_Rx_mean,pose_Rx_max,pose_Rx_std,pose_Ry_mean,pose_Ry_max,pose_Ry_std,pose_Rz_mean,pose_Rz_max,pose_Rz_std,gaze_angle_x_mean,gaze_angle_x_max,gaze_angle_x_std,gaze_angle_y_mean,gaze_angle_y_max,gaze_angle_y_std
0,2024-12-01 22:55:00,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.675033,-0.638,0.009242,0.179933,0.189,0.002778,0.128667,0.129,0.001155,0.000000,0.000,0.000000,0.000000,0.000,0.000000
1,2024-12-01 22:55:01,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.678000,-0.678,0.000000,0.179000,0.179,0.000000,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000
2,2024-12-01 22:55:02,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,-0.678000,-0.678,0.000000,0.179000,0.179,0.000000,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000
3,2024-12-01 22:55:03,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0,0.0,0.000000,0.00,0.000000

In [1]:
from helper.au_extract import get_au_data

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

start_time = '2024-12-01 22-55-00'
end_time = '2024-12-01 23-02-00'
merged_df = get_au_data(start_time, end_time, '1S')



In [3]:
#merged df should have timestamp, but this data is in +5:30 zone, even if its not mentioned
merged_df['timestamp'] = merged_df['absolute_timestamp'].dt.strftime('%Y-%m-%d %H-%M-%S')
merged_df

,absolute_timestamp,AU01_r_mean,AU01_r_max,AU01_r_std,AU02_r_mean,AU02_r_max,AU02_r_std,AU04_r_mean,AU04_r_max,AU04_r_std,...,pose_Rz_mean,pose_Rz_max,pose_Rz_std,gaze_angle_x_mean,gaze_angle_x_max,gaze_angle_x_std,gaze_angle_y_mean,gaze_angle_y_max,gaze_angle_y_std,timestamp
0,2024-12-01 22:55:00,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.128667,0.129,0.001155,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-00
1,2024-12-01 22:55:01,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-01
2,2024-12-01 22:55:02,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-02
3,2024-12-01 22:55:03,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-03
4,2024-12-01 22:55:04,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2024-12-01 23:01:56,0.015,0.08,0.028009,0.015000,0.15,0.042325,0.389333,0.98,0.353484,...,-0.141033,-0.068,0.042416,-0.111767,-0.004,0.051818,0.119833,0.268,0.042795,2024-12-01 23-01-56
417,2024-12-01 23:01:57,0.000,0.00,0.000000,0.001000,0.01,0.003051,0.061333,0.22,0.068165,...,-0.065367,-0.055,0.005792,-0.046733,0.011,0.033204,0.014067,0.229,0.087650,2024-12-01 23-01-57
418,2024-12-01 23:01:58,0.000,0.00,0.000000,0.105667,0.74,0.203481,0.233000,0.94,0.361674,...,-0.112500,-0.041,0.071546,-0.020767,0.007,0.015822,0.035267,0.064,0.024036,2024-12-01 23-01-58
419,2024-12-01 23:01:59,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.772333,1.03,0.109409,...,-0.187967,-0.143,0.032496,0.035033,0.084,0.033560,0.019533,0.056,0.021365,2024-12-01 23-01-59


In [4]:
from helper.hr_analysis import extract_hr_features


In [5]:
# Define start and end times in the specified format
# start_time = '2024-12-01 22-55-00'
# end_time = '2024-12-01 23-02-00'
start_time = '2024-12-01 10:55:00 PM +0530'
end_time = '2024-12-01 11:02:00 PM +0530'

# Define granularity in seconds (e.g., 60 for 1-minute segments)
granularity = 5

# Call the extract_hr_features function
features = extract_hr_features(start_time, end_time, granularity)
features

/teamspace/studios/this_studio/helper/hr_analysis.py:120: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'], errors='coerce')


,Segment Start,Mean HR,HR Slope,HR Variability,Peak HR,HR Change,Moving Average HR,HR Transitions
0,2024-12-01 22:55:55+05:30,70.0,0,0.0,70.0,0.0,70.0,0
1,2024-12-01 22:56:00+05:30,66.0,0,0.0,66.0,-4.0,66.0,0
2,2024-12-01 22:56:05+05:30,64.0,0,0.0,64.0,-2.0,64.0,0
3,2024-12-01 22:56:10+05:30,61.0,0,0.0,61.0,-3.0,61.0,0
4,2024-12-01 22:56:15+05:30,63.0,0,0.0,63.0,2.0,63.0,0
...,...,...,...,...,...,...,...,...
68,2024-12-01 23:01:35+05:30,67.0,0,0.0,67.0,0.0,67.0,0
69,2024-12-01 23:01:40+05:30,72.0,0,0.0,72.0,5.0,72.0,0
70,2024-12-01 23:01:45+05:30,70.0,0,0.0,70.0,-2.0,70.0,0
71,2024-12-01 23:01:50+05:30,69.0,0,0.0,69.0,-1.0,69.0,0


In [6]:
features.to_csv('features.csv')

In [7]:
merged_df.to_csv('merged_df.csv')

In [8]:
c_features = features.copy()
c_merged_df = merged_df.copy()

In [9]:
# Convert timestamps to datetime format
features['Segment Start'] = pd.to_datetime(features['Segment Start'], errors='coerce').dt.tz_convert('UTC')
merged_df['absolute_timestamp'] = pd.to_datetime(merged_df['absolute_timestamp'], errors='coerce').dt.tz_localize('UTC')

# Add 5 hours and 30 minutes to the 'Segment Start' column for alignment
features['Segment Start'] = features['Segment Start'] + pd.Timedelta(hours=5, minutes=30)
# convert 5:30 to UTC format
features['Segment Start'] = features['Segment Start']



In [10]:
# Merge the dataframes on the adjusted timestamps
merged_data = pd.merge(
    features,
    merged_df,
    how='right',
    left_on='Segment Start',
    right_on='absolute_timestamp',
)

In [11]:
merged_data

,Segment Start,Mean HR,HR Slope,HR Variability,Peak HR,HR Change,Moving Average HR,HR Transitions,absolute_timestamp,AU01_r_mean,...,pose_Rz_mean,pose_Rz_max,pose_Rz_std,gaze_angle_x_mean,gaze_angle_x_max,gaze_angle_x_std,gaze_angle_y_mean,gaze_angle_y_max,gaze_angle_y_std,timestamp
0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 22:55:00+00:00,0.000,...,0.128667,0.129,0.001155,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-00
1,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 22:55:01+00:00,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-01
2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 22:55:02+00:00,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-02
3,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 22:55:03+00:00,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-03
4,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 22:55:04+00:00,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 23:01:56+00:00,0.015,...,-0.141033,-0.068,0.042416,-0.111767,-0.004,0.051818,0.119833,0.268,0.042795,2024-12-01 23-01-56
417,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 23:01:57+00:00,0.000,...,-0.065367,-0.055,0.005792,-0.046733,0.011,0.033204,0.014067,0.229,0.087650,2024-12-01 23-01-57
418,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 23:01:58+00:00,0.000,...,-0.112500,-0.041,0.071546,-0.020767,0.007,0.015822,0.035267,0.064,0.024036,2024-12-01 23-01-58
419,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-12-01 23:01:59+00:00,0.000,...,-0.187967,-0.143,0.032496,0.035033,0.084,0.033560,0.019533,0.056,0.021365,2024-12-01 23-01-59


In [12]:
adjusted_merged_data = merged_data.copy()
adjusted_merged_data.sort_values(by='absolute_timestamp', inplace=True)

# Set 'absolute_timestamp' as the index for interpolation
adjusted_merged_data.set_index('absolute_timestamp', inplace=True)

# Interpolate missing data with a time limit of 2.5 seconds
adjusted_merged_data = adjusted_merged_data.interpolate(method='linear', limit=3, limit_direction='both')

# Reset the index back to default
adjusted_merged_data.reset_index(inplace=True)

adjusted_merged_data

/tmp/ipykernel_5272/1501095696.py:8: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  adjusted_merged_data = adjusted_merged_data.interpolate(method='linear', limit=3, limit_direction='both')


,absolute_timestamp,Segment Start,Mean HR,HR Slope,HR Variability,Peak HR,HR Change,Moving Average HR,HR Transitions,AU01_r_mean,...,pose_Rz_mean,pose_Rz_max,pose_Rz_std,gaze_angle_x_mean,gaze_angle_x_max,gaze_angle_x_std,gaze_angle_y_mean,gaze_angle_y_max,gaze_angle_y_std,timestamp
0,2024-12-01 22:55:00+00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,...,0.128667,0.129,0.001155,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-00
1,2024-12-01 22:55:01+00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-01
2,2024-12-01 22:55:02+00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-02
3,2024-12-01 22:55:03+00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-03
4,2024-12-01 22:55:04+00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,...,0.129000,0.129,0.000000,0.000000,0.000,0.000000,0.000000,0.000,0.000000,2024-12-01 22-55-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,2024-12-01 23:01:56+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.015,...,-0.141033,-0.068,0.042416,-0.111767,-0.004,0.051818,0.119833,0.268,0.042795,2024-12-01 23-01-56
417,2024-12-01 23:01:57+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.000,...,-0.065367,-0.055,0.005792,-0.046733,0.011,0.033204,0.014067,0.229,0.087650,2024-12-01 23-01-57
418,2024-12-01 23:01:58+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.000,...,-0.112500,-0.041,0.071546,-0.020767,0.007,0.015822,0.035267,0.064,0.024036,2024-12-01 23-01-58
419,2024-12-01 23:01:59+00:00,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,...,-0.187967,-0.143,0.032496,0.035033,0.084,0.033560,0.019533,0.056,0.021365,2024-12-01 23-01-59


In [13]:
list(adjusted_merged_data.columns)

['absolute_timestamp',
 'Segment Start',
 'Mean HR',
 'HR Slope',
 'HR Variability',
 'Peak HR',
 'HR Change',
 'Moving Average HR',
 'HR Transitions',
 'AU01_r_mean',
 'AU01_r_max',
 'AU01_r_std',
 'AU02_r_mean',
 'AU02_r_max',
 'AU02_r_std',
 'AU04_r_mean',
 'AU04_r_max',
 'AU04_r_std',
 'AU05_r_mean',
 'AU05_r_max',
 'AU05_r_std',
 'AU06_r_mean',
 'AU06_r_max',
 'AU06_r_std',
 'AU07_r_mean',
 'AU07_r_max',
 'AU07_r_std',
 'AU09_r_mean',
 'AU09_r_max',
 'AU09_r_std',
 'AU10_r_mean',
 'AU10_r_max',
 'AU10_r_std',
 'AU12_r_mean',
 'AU12_r_max',
 'AU12_r_std',
 'AU14_r_mean',
 'AU14_r_max',
 'AU14_r_std',
 'AU15_r_mean',
 'AU15_r_max',
 'AU15_r_std',
 'AU17_r_mean',
 'AU17_r_max',
 'AU17_r_std',
 'AU20_r_mean',
 'AU20_r_max',
 'AU20_r_std',
 'AU23_r_mean',
 'AU23_r_max',
 'AU23_r_std',
 'AU25_r_mean',
 'AU25_r_max',
 'AU25_r_std',
 'AU26_r_mean',
 'AU26_r_max',
 'AU26_r_std',
 'AU45_r_mean',
 'AU45_r_max',
 'AU45_r_std',
 'AU01_c_mean',
 'AU01_c_max',
 'AU01_c_std',
 'AU02_c_mean',
 'AU

In [14]:
# drop based on MeanHR
adjusted_merged_data = adjusted_merged_data.dropna(subset=['Mean HR'])
adjusted_merged_data

,absolute_timestamp,Segment Start,Mean HR,HR Slope,HR Variability,Peak HR,HR Change,Moving Average HR,HR Transitions,AU01_r_mean,...,pose_Rz_mean,pose_Rz_max,pose_Rz_std,gaze_angle_x_mean,gaze_angle_x_max,gaze_angle_x_std,gaze_angle_y_mean,gaze_angle_y_max,gaze_angle_y_std,timestamp
52,2024-12-01 22:55:52+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.006667,...,-0.055667,-0.006,0.057119,0.213800,0.365,0.137429,0.101400,0.185,0.038063,2024-12-01 22-55-52
53,2024-12-01 22:55:53+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.240000,...,-0.099867,-0.087,0.010071,0.016733,0.062,0.017033,0.048500,0.083,0.018446,2024-12-01 22-55-53
54,2024-12-01 22:55:54+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.112667,...,-0.038067,-0.013,0.017380,-0.010167,0.020,0.015559,0.067167,0.238,0.043496,2024-12-01 22-55-54
55,2024-12-01 22:55:55+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.108667,...,-0.051600,-0.041,0.005975,-0.023733,0.100,0.042869,0.061700,0.240,0.062369,2024-12-01 22-55-55
56,2024-12-01 22:55:56+00:00,2024-12-01 22:55:56+00:00,69.2,0.0,0.0,69.2,-0.8,69.2,0.0,0.177000,...,-0.019600,0.086,0.048073,0.093300,0.280,0.117528,0.047033,0.135,0.051462,2024-12-01 22-55-56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,2024-12-01 23:01:54+00:00,2024-12-01 23:01:54+00:00,65.0,0.0,0.0,65.0,-4.2,65.0,0.0,0.000000,...,-0.106400,-0.093,0.011016,-0.110000,-0.063,0.022362,-0.007000,0.036,0.023068,2024-12-01 23-01-54
415,2024-12-01 23:01:55+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.001000,...,-0.117800,-0.095,0.020326,-0.087367,-0.046,0.028282,0.079767,0.137,0.033689,2024-12-01 23-01-55
416,2024-12-01 23:01:56+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.015000,...,-0.141033,-0.068,0.042416,-0.111767,-0.004,0.051818,0.119833,0.268,0.042795,2024-12-01 23-01-56
417,2024-12-01 23:01:57+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.000000,...,-0.065367,-0.055,0.005792,-0.046733,0.011,0.033204,0.014067,0.229,0.087650,2024-12-01 23-01-57


In [17]:


merged_df = adjusted_merged_data.copy()
# 1. Composite Emotional Scores
merged_df['Positive_Engagement'] = (merged_df['AU06_r_mean'] + merged_df['AU12_r_mean'] + merged_df['AU14_r_mean']) - (merged_df['AU04_r_mean'] + merged_df['AU07_r_mean'])
merged_df['Stress_Score'] = (merged_df['AU04_r_mean'] + merged_df['AU07_r_mean'] + merged_df['AU23_r_mean']) - (merged_df['AU06_r_mean'] + merged_df['AU12_r_mean'])

# 2. Happiness-to-Stress Ratio
merged_df['Happiness_Stress_Ratio'] = (merged_df['AU06_r_mean'] + merged_df['AU12_r_mean']) / (merged_df['AU04_r_mean'] + merged_df['AU07_r_mean'] + merged_df['AU23_r_mean'] + 1e-5)


# 4. Engagement Score
merged_df['Engagement_Score'] = merged_df['HR Variability'] * merged_df['AU12_r_mean']

# 5. Head Movement Intensity
merged_df['Head_Movement_Intensity'] = np.sqrt(merged_df['pose_Rx_std']**2 + merged_df['pose_Ry_std']**2 + merged_df['pose_Rz_std']**2)

# 6. Gaze Consistency
merged_df['Gaze_Consistency'] = merged_df[['gaze_angle_x_mean', 'gaze_angle_y_mean']].var(axis=1)

# 7. Valence and Arousal
merged_df['Valence'] = (merged_df['AU06_r_mean'] + merged_df['AU12_r_mean'] + merged_df['AU14_r_mean']) / 3
merged_df['Arousal'] = (merged_df['AU04_r_mean'] + merged_df['AU07_r_mean'] + merged_df['AU09_r_mean'] +
                        merged_df['AU10_r_mean'] + merged_df['AU20_r_mean'] + merged_df['AU23_r_mean'] +
                        merged_df['AU25_r_mean'] + merged_df['AU26_r_mean']) / 8

# 8. Facial Activity Variability (AU Peak Ratio)
merged_df['AU_Peak_Ratio'] = (merged_df['AU06_r_mean'] + merged_df['AU12_r_mean']) / (merged_df['AU04_r_mean'] + merged_df['AU07_r_mean'] + 1e-5)

# 9. PCA on Selected AUs
pca_features = ['AU01_r_mean', 'AU02_r_mean', 'AU04_r_mean', 'AU05_r_mean', 'AU06_r_mean',
               'AU07_r_mean', 'AU09_r_mean', 'AU10_r_mean', 'AU12_r_mean', 'AU14_r_mean',
               'AU15_r_mean', 'AU17_r_mean', 'AU20_r_mean', 'AU23_r_mean', 'AU25_r_mean',
               'AU26_r_mean', 'AU45_r_mean']

scaler = StandardScaler()
scaled_pca_features = scaler.fit_transform(merged_df[pca_features])

pca = PCA(n_components=5)
principal_components = pca.fit_transform(scaled_pca_features)

for i in range(1, 6):
    merged_df[f'PC{i}'] = principal_components[:, i-1]
# 3. Cognitive Load Index
merged_df['Cognitive_Load'] = (
    0.10 * merged_df['Mean HR'] +
    0.10 * merged_df['HR Variability'] +
    0.15 * merged_df['AU04_r_mean'] +
    0.15 * merged_df['AU07_r_mean'] +
    0.10 * merged_df['Stress_Score'] +
    0.05 * merged_df['Arousal'] +
    0.05 * merged_df['Valence'] +
    0.05 * merged_df['Head_Movement_Intensity'] +
    0.05 * merged_df['Gaze_Consistency']
)


# 10. Final Feature Set
final_features = [
    'Positive_Engagement', 'Stress_Score', 'Happiness_Stress_Ratio',
    'Cognitive_Load', 'Engagement_Score', 'Head_Movement_Intensity',
    'Gaze_Consistency', 'Valence', 'Arousal', 'AU_Peak_Ratio',
    'PC1', 'PC2', 'PC3', 'PC4', 'PC5'
]


# Optionally, drop original AU columns to reduce dimensionality
merged_df = merged_df.drop(columns=pca_features)

# Now, `merged_df` contains all the engineered features ready for modeling
merged_df

,absolute_timestamp,Segment Start,Mean HR,HR Slope,HR Variability,Peak HR,HR Change,Moving Average HR,HR Transitions,AU01_r_max,...,Gaze_Consistency,Valence,Arousal,AU_Peak_Ratio,PC1,PC2,PC3,PC4,PC5,Cognitive_Load
52,2024-12-01 22:55:52+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.10,...,0.006317,0.874000,1.018792,0.965596,-0.627910,1.753933,1.305588,-0.899885,0.652236,7.356789
53,2024-12-01 22:55:53+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.43,...,0.000505,0.413889,0.860000,0.391939,-0.963446,0.220680,-1.328002,0.067802,0.337163,7.732587
54,2024-12-01 22:55:54+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.34,...,0.002990,0.365111,0.832583,0.348275,-1.045504,-0.040283,-1.282887,0.026888,0.603760,7.727103
55,2024-12-01 22:55:55+00:00,2024-12-01 22:55:55+00:00,70.0,0.0,0.0,70.0,0.0,70.0,0.0,0.56,...,0.003649,0.215556,0.346708,0.326100,1.765288,-1.545952,-0.307822,-0.945137,0.824724,7.360740
56,2024-12-01 22:55:56+00:00,2024-12-01 22:55:56+00:00,69.2,0.0,0.0,69.2,-0.8,69.2,0.0,0.53,...,0.001070,0.318556,0.305708,0.380067,2.269342,-1.095883,-0.201874,-1.571513,-0.041825,7.167156
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,2024-12-01 23:01:54+00:00,2024-12-01 23:01:54+00:00,65.0,0.0,0.0,65.0,-4.2,65.0,0.0,0.00,...,0.005305,0.261444,0.651208,0.036116,0.936153,-1.610280,-0.698997,1.851900,-0.419900,7.457619
415,2024-12-01 23:01:55+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.01,...,0.013967,0.275444,0.708167,0.039697,1.039375,-1.677469,-0.769860,1.571585,0.041317,7.409491
416,2024-12-01 23:01:56+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.08,...,0.026819,0.364000,0.649042,0.101706,1.326375,-1.087752,-0.458077,0.467403,2.175610,7.167719
417,2024-12-01 23:01:57+00:00,2024-12-01 23:01:55+00:00,64.0,0.0,0.0,64.0,-5.0,64.0,0.0,0.00,...,0.001848,0.348222,0.885000,0.150538,2.827083,-1.727554,0.233053,2.566597,0.368716,7.292814


In [ ]:
final_features